In [3]:
import os
import numpy as np
from gmsh_mesh_builder import Mesher
import gmsh  # ! remember to remove this line

cwd = os.getcwd()
geo_file_path = f"{cwd}/04_OUTPUT/C0002237/fake_example.geo_unrolled"
mesh_file_path = f"{cwd}/04_OUTPUT/C0002237/C0002237.msh"
idx_cort_int_path = "/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/04_OUTPUT/C0002237/C0002237_intersections_int.npy"
coi_idx = np.load(idx_cort_int_path)
trabecular_volume = Mesher(geo_file_path, mesh_file_path, slicing_coefficient=6)

In [4]:
def principal_axes_length(array):
    l_i = np.linalg.norm(array[0] - array[1])
    l_j = np.linalg.norm(array[0] - array[2])
    return l_i, l_j

def get_offset_points(array, _center, l_i, l_j, LENGTH_FACTOR):
    # calculate offset point from _center and 0.6 * half of the principal axes length
    OFFSET = LENGTH_FACTOR
    offset_i = [OFFSET * l_i / 2, 0, 0]
    offset_j = [OFFSET * 0, l_j / 2, 0]
    
    # replace the offset point in the array
    array[0] = np.array(_center) + np.array(offset_i)
    array[1] = np.array(_center) - np.array(offset_i)
    array[2] = np.array(_center) + np.array(offset_j)
    array[3] = np.array(_center) - np.array(offset_j)
    return array

def get_trabecular_position(coi_idx, LENGTH_FACTOR):
    coi_idx_r = np.reshape(coi_idx, (-1, 3))
    # create subarrays of the coi_idx array for each slice (coi_idx[:, 2])
    coi_idx_every_4_points = np.split(coi_idx_r, np.where(np.diff(coi_idx_r[:, 2]))[0]+1)

    # iterate over the subarrays and calculate the principal axes length
    trabecular_points = np.empty((len(coi_idx_every_4_points), 4, 3))
    for i, _ in enumerate(coi_idx_every_4_points):
        c_x = coi_idx_every_4_points[i][:, 0]
        c_y = coi_idx_every_4_points[i][:, 1]
        c_z = coi_idx_every_4_points[i][:, 2]
        _center = [np.mean(c_x), np.mean(c_y), np.mean(c_z)]

        # calculate the principal axes length
        l_i, l_j = principal_axes_length(coi_idx_every_4_points[i])
        trabecular_points[i] = get_offset_points(coi_idx_every_4_points[i], _center, l_i, l_j, LENGTH_FACTOR)
        
        # sort points in cw direction
        trabecular_points[i] = trabecular_points[i][[0, 2, 1, 3]]

    return np.array(trabecular_points, dtype=np.float32).reshape((-1, 3))

def trabecular_mesh():
    gmsh.initialize()  # ! remember to remove this line
    gmsh.clear()  # ! remember to remove this line
    trabecular_points = get_trabecular_position(coi_idx, 0.6)
    point_tags = trabecular_volume.insert_points(trabecular_points)
    print(point_tags)
    point_tags_r = np.reshape(point_tags, (-1, 4))
    # concatenate first point to the end of each subarray
    
    points_first_column = point_tags_r[:, 0]
    point_tags_c = np.concatenate((point_tags_r, points_first_column[:, None]), axis=1)
    

    line_tags_h = []
    for i in range(len(point_tags_c[:, 0])):
        line_tags_s = trabecular_volume.insert_lines(point_tags_c[i])
        line_tags_h.append(line_tags_s)
    
    surf_tags_h = []
    for i, _ in enumerate(line_tags_h):
        trab_curveloop_h = trabecular_volume.factory.addCurveLoop(line_tags_h[i], tag=-1)
        trab_tag_h = trabecular_volume.factory.addPlaneSurface([trab_curveloop_h], tag=-1)
        surf_tags_h.append(trab_tag_h)
    
    line_tags_v = []
    for j in range(len(point_tags_c[0, :])-1):
        line_tags_s = trabecular_volume.insert_lines(point_tags_c[:, j])
        line_tags_v.append(line_tags_s)
        
    line_tags_v = np.array(line_tags_v, dtype=int).reshape((-1, 5))
    line_tags_v = np.concatenate((line_tags_v, line_tags_v[:, 0][:, None]), axis=1)
    line_tags_v = np.append(line_tags_v, line_tags_v[0, :][None, :], axis=0).tolist()
    
    line_tags_h = np.array(line_tags_h, dtype=int).reshape((-1, 4))
    line_tags_h = np.concatenate((line_tags_h, line_tags_h[:, 0][:, None]), axis=1).tolist()

    surf_tags_v = []
    trab_surf_loop_tag = []
    for j in range(len(line_tags_v)-1):
        line_tag = line_tags_v[j]
        for i in range(len(line_tag)-1):
            trab_curveloop_v = trabecular_volume.factory.addCurveLoop([line_tags_h[i][j], line_tags_v[j][i], line_tags_h[i+1][j], line_tags_v[j+1][i]], tag=-1)
            trab_tag_v = trabecular_volume.factory.addSurfaceFilling(trab_curveloop_v, tag=-1)
            surf_tags_v.append(trab_tag_v)
            # trab_surf_loop_tag = trabecular_volume.factory.addSurfaceLoop(surf_tags_v, tag=-1)
            trab_surf_loop_s = trabecular_volume.factory.addSurfaceLoop([trab_tag_v], tag=-1)
            trab_surf_loop_tag.append(trab_surf_loop_s)

    # make volume
    
    trab_vol_tag = []
    for i in range(len(trab_surf_loop_tag)):
        volume_t = trabecular_volume.factory.addVolume([trab_surf_loop_tag[i]], tag=-1)
        trab_vol_tag.append(volume_t)
    
    # trab_vol_tag = trabecular_volume.factory.addVolume([trab_surf_loop_tag], tag=-1)

    line_tags = np.append(np.unique(line_tags_h), np.unique(line_tags_v))
    line_tags = list(map(int, line_tags))
    surf_tags = np.append(np.unique(surf_tags_h), np.unique(surf_tags_v))
    surf_tags = list(map(int, surf_tags))
    vol_tags = list(map(int, trab_vol_tag))
    
    n_transverse = 5
    n_radial = 7
    
    # make transfinite
    trabecular_volume.factory.synchronize()
    for line in line_tags:
        trabecular_volume.model.mesh.setTransfiniteCurve(line, n_radial)
    for surface in surf_tags:
        trabecular_volume.model.mesh.setTransfiniteSurface(surface)
    for volume in vol_tags:
        trabecular_volume.model.mesh.setTransfiniteVolume(volume)
    
    trabecular_volume.mesh_generate(dim=2)
    gmsh.fltk.run()
    gmsh.finalize()


trabecular_mesh()


Info    : Clearing all models and views...
Info    : Done clearing all models and views
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 30%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Inf